In [8]:
import numpy
import random

In [2]:
def copying_data(T=30, n_data=300*64, n_sequence=10, batch_size=64, make_rand=False, shuffle=False, drop_last=False):
    seq = numpy.random.randint(1, high=9, size=(n_data, n_sequence))
    zeros1 = numpy.zeros((n_data, T-1))
    zeros2 = numpy.zeros((n_data, T))
    marker = 9 * numpy.ones((n_data, 1))
    zeros3 = numpy.zeros((n_data, n_sequence))

    x = numpy.concatenate((seq, zeros1, marker, zeros3), axis=1).astype('int32')
    y = numpy.concatenate((zeros3, zeros2, seq), axis=1).astype('int64')

    x = numpy.expand_dims(x, axis=-1)
    y = numpy.expand_dims(y, axis=-1)
    
    return x, y

In [3]:
random.seed(1)

train_x, train_y = copying_data(T=50)
test_x, test_y   = copying_data(T=200)

# Model comparison

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import LSTMCell
from tfRIM import RIMCell

## LSTM model

In [9]:
tf.random.set_seed(1234)
lstmcell = LSTMCell(600)

In [4]:
inputs = tf.keras.Input([None, 1])
rnnout = tf.keras.layers.RNN(lstmcell, return_sequences=True)(inputs)
seqout = tf.keras.layers.Dense(10, activation='softmax')(rnnout)
output = seqout[:, -10:]
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
cost = tf.keras.losses.SparseCategoricalCrossentropy()

model_lstm = tf.keras.Model(inputs, output)
model_lstm.compile(optimizer=opt, loss=cost)

In [ ]:
tf.random.set_seed(1234)
model_lstm.fit(train_x, train_y, batch_size=128, epochs=100) #Be careful, this takes time on your laptop CPU...

## RIMs model

In [10]:
tf.random.set_seed(1234)
rimcell = RIMCell(units=100, nRIM=6, k=4,
                num_input_heads=1, input_key_size=32, input_value_size=32, input_query_size=32, input_keep_prob=0.9,
                num_comm_heads=4, comm_key_size=32, comm_value_size=32, comm_query_size=32, comm_keep_prob=0.9)

In [6]:
inputs = tf.keras.Input([None, 1])
rnnout = tf.keras.layers.RNN(rimcell, return_sequences=True)(inputs)
seqout = tf.keras.layers.Dense(10, activation='softmax')(rnnout)
output = seqout[:, -10:]
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
cost = tf.keras.losses.SparseCategoricalCrossentropy()

model_rims = tf.keras.Model(inputs, output)
model_rims.compile(optimizer=opt, loss=cost)

In [ ]:
tf.random.set_seed(1234)
model_rims.fit(train_x, train_y, batch_size=128, epochs=100) #Be careful, this takes time on your laptop CPU...

## Evaluate

#### In sample prediction

In [ ]:
ypred1_lstm = model_lstm.predict(train_x)
ypred1_rims = model_rims.predict(train_x)

cost = tf.keras.losses.SparseCategoricalCrossentropy()

ce_train_lstm = cost(train_y[:,-10:], ypred1_lstm)
ce_train_rims = cost(train_y[:,-10:], ypred1_rims)

ac_train_lstm = (numpy.argmax(ypred1_lstm, axis=-1) == train_y[:,-10:,0]).mean()
ac_train_rims = (numpy.argmax(ypred1_rims, axis=-1) == train_y[:,-10:,0]).mean()

In [ ]:
ce_train_lstm, ce_train_rims

In [ ]:
ac_train_lstm, ac_train_rims

#### Out of sample prediction

In [ ]:
ypred2_lstm = model_lstm.predict(test_x)
ypred2_rims = model_rims.predict(test_x)

cost = tf.keras.losses.SparseCategoricalCrossentropy()

ce_test_lstm = cost(test_y[:,-10:], ypred2_lstm)
ce_test_rims = cost(test_y[:,-10:], ypred2_rims)

ac_test_lstm = (numpy.argmax(ypred2_lstm, axis=-1) == test_y[:,-10:,0]).mean()
ac_test_rims = (numpy.argmax(ypred2_rims, axis=-1) == test_y[:,-10:,0]).mean()

In [7]:
ce_test_lstm, ce_test_rims

In [ ]:
ac_test_lstm, ac_test_rims